### Introduction to Shapely

https://shapely.readthedocs.io/en/stable/

Shapely is a BSD-licensed Python package for **manipulation** and **analysis** of **planar geometric objects**. 

* Shapely is **not** concerned with data formats or coordinate systems.
* Shapely is based on the widely deployed GEOS (the engine of PostGIS) and JTS (from which GEOS is ported) libraries.

### Simple Feature Access

http://www.opengeospatial.org/standards/sfa

https://en.wikipedia.org/wiki/Simple_Features


**Simple Feature Access** is both an Open Geospatial Consortium (OGC) and International Organization for Standardization (ISO) standard **ISO 19125** that specifies a common storage and access model of mostly two-dimensional geometries (point, line, polygon, multi-point, multi-line, etc.) used by geographic information systems.

Shapely supports the following Features:

| Geometry            | Description  |
| ------------------- | -------------|
|Point                | A single coordinate with x,y and possibly z values | 
|LineString           | One or more line segments |
|LinearRing           | One or more line segments that forms a closed loop |
|Polygon              | An area that is enclosed by a linear ring |
|MultiPoint           | A collection of one or more Points |
|MultiLineString      | A collection of one or more LineStrings |
|MultiPolygon         | A collection of one or more Polygons |
|GeometryCollection   | A collection of one or more geometries that may contain more than one type of geometry |



## Operations with Polygon / MutliPolygon

In [ ]:
from shapely.geometry import Polygon, Point, MultiPolygon

polygon1 = Polygon([(30, 10), (40, 40), (20, 35), (10, 20), (30, 10)])

print(f"Polygon area: {polygon1.area}, polygon length: {polygon1.length}") 

In [ ]:
polygon1

In [ ]:
polygon2 = Polygon([(20,20),(80,30),(50,40),(20,20)])
polygon2

In [ ]:
polygon2.union(polygon1)

In [ ]:
polygon2.intersection(polygon1)

In [ ]:
polygon2.symmetric_difference(polygon1)

In [ ]:
result = polygon2.symmetric_difference(polygon1)

print(f"Polygon area: {result.area}, polygon length: {result.length}")

In [ ]:
result.wkt # well known text

In [ ]:
s = result.wkt
type(s)

In [ ]:
s = 'MULTIPOLYGON (((20 20, 34.11764705882353 22.35294117647059, 30 10, 10 20, 20 35, 40 40, 37.14285714285715 31.42857142857143, 20 20)), ((37.14285714285715 31.42857142857143, 50 40, 80 30, 34.11764705882353 22.35294117647059, 37.14285714285715 31.42857142857143)))'

In [ ]:
import shapely.wkt

mypolygon = shapely.wkt.loads(s)
mypolygon

There are also several binary operations available:

- **contains** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **intersects** (Returns True if the boundary and interior of the object intersect in any way with those of the other.)
- **witin** (Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior).
- **touches** (Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other.)
- **crosses** (Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.)
- **equals** (Returns True if the set-theoretic boundary, interior, and exterior of the object coincide with those of the other.)

In [ ]:
polygon1.intersects(polygon2)

In [ ]:
polygon1.within(polygon2)

In [ ]:
polygon1.equals(polygon1)

## Reading Vector Files using Fiona

https://github.com/Toblerity/Fiona

Spatial information is not only geometry. Together with it we always have metadata (properties). Lets look at a first dataset.


We're using the GeoPackage file format. To list all available layers:
     
    filename = "geodata/packages/natural_earth_vector.gpkg"
    layers = fiona.listlayers(filename)
    print(layers)


In [ ]:
import fiona

fiona.supported_drivers

In [ ]:
if 'GPKG' in fiona.supported_drivers:
    print("GeoPackage is supported...")

In [ ]:
import fiona

c = fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports')

airport = next(iter(c))
airport

In [ ]:
airport['properties']['name']

In [ ]:
airport['geometry']['type']

In [ ]:
airport['geometry']['coordinates']

In [ ]:
c.close()

Lets read all data. There are basically two ways:

a) load everything to memory: (if dataset isn't too large...)

    alldata = list(c)
    
b) iterate through all data: (one by one):

    for element in c:
        ...

In [ ]:
with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports') as c:
    for airport in c:
        if airport['properties']['iata_code'] == "ZRH":
            #print(airport['properties']['name'])
            #print(airport['geometry']['coordinates'])
            #print(airport['properties']['wikipedia'])
            print(airport)

In [ ]:
with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports') as c:
    print(c.crs)

In [ ]:
import fiona

c = fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_admin_0_countries')

country = next(iter(c))

print(country['properties']['NAME'])
print(country['properties']['NAME_ZH'])
print(country['properties']['CONTINENT'])
print(country['properties']['POP_EST'])
print(country['properties']['POP_YEAR'])


In [ ]:
with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_admin_0_countries') as c:
    for country in c:
        if country['properties']['NAME'] == "Switzerland":
            print(country['properties']['POP_EST'])
            print(country['properties']['POP_YEAR'])   
            print(country['geometry']['type'])
            # print(country['geometry']['coordinates']) # you don't want to print all coordinates!
            geomtype = country['geometry']['type']
            geometry = country['geometry']


## Draw Shapely Polygon using cartopy

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

Previously we had this shape:

In [ ]:
result

Now let's display this in Cartopy using PlateCarree:

Note, that we need a projection to use Plate Carree.

In [ ]:
from cartopy.feature import ShapelyFeature
from shapely.geometry import shape

proj = ccrs.PlateCarree()

ax = plt.axes(projection=proj)
ax.set_extent((result.bounds[0], result.bounds[2], result.bounds[1], result.bounds[3]), crs=ccrs.PlateCarree())
shape_feature = ShapelyFeature([result], ccrs.PlateCarree(), facecolor='#AAFFAA', edgecolor='k')
ax.add_feature(shape_feature);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.1, linestyle='--')
gl.xlabels_top = False
gl.ylabels_left = False
gl.xlabel_style = {'size': 10, 'color': 'black'}
gl.ylabel_style = {'size': 10, 'color': 'black'}


Let's plot the Switzerland Polygon using Cartopy:

In [ ]:
from cartopy.feature import ShapelyFeature
from shapely.geometry import shape
from shapely.geometry import Polygon

ch = Polygon(shape(geometry))

######################
proj = ccrs.Mercator()
######################

ax = plt.axes(projection=proj)
ax.set_extent((ch.bounds[0], ch.bounds[2], ch.bounds[1], ch.bounds[3]), crs=ccrs.PlateCarree())
shape_feature = ShapelyFeature([ch], ccrs.PlateCarree(), facecolor='#AAFFAA', edgecolor='k')
ax.add_feature(shape_feature)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='black', alpha=0.5, linestyle='--')

In [ ]:
proj = ccrs.PlateCarree()

ax = plt.axes(projection=proj)
ax.set_extent((ch.bounds[0], ch.bounds[2], ch.bounds[1], ch.bounds[3]), crs=ccrs.PlateCarree())
shape_feature = ShapelyFeature([ch], ccrs.PlateCarree(), facecolor='#AAFFAA', edgecolor='k')
ax.add_feature(shape_feature);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')

In [ ]:
x = [8.539418]
y = [47.378115]  # Zurich, Switzerland

proj = ccrs.Mercator()

plt.figure(figsize=(15, 9))
ax = plt.axes(projection=proj)
ax.set_extent((ch.bounds[0], ch.bounds[2], ch.bounds[1], ch.bounds[3]), crs=ccrs.PlateCarree())
shape_feature = ShapelyFeature([ch], ccrs.PlateCarree(), facecolor='#EEFFFF', edgecolor='k')
ax.add_feature(shape_feature);
ax.plot(x,y, color='blue', linewidth=2, marker='o', transform=ccrs.Geodetic())
plt.text(x[0], y[0]*1.0005, 
         "Zürich", 
         horizontalalignment='center', 
         weight="regular", 
         color="blue", 
         fontsize=10, 
         transform=ccrs.Geodetic());
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')

## Find Airports inside the polygon

First we convert it to a shapely Polygon. If you use another country, it may be a Multipolygon, check the output above!

In [ ]:
from shapely.geometry import shape
from shapely.geometry import Polygon   # oder MultiPolygon für andere Länder

ch = Polygon(shape(geometry))
ch

In [ ]:
from shapely.geometry import Point

with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports') as c:
    for airport in c:      
        position = Point(airport['geometry']['coordinates'])
        if position.within(ch):
            print(airport['properties']['iata_code'], airport['properties']['name'], airport['geometry']['coordinates'])

### Find Areas with a population greater than 75'000 inside the Polygon

In [ ]:
import fiona

cnt = 0
with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_populated_places') as c:
    for place in c:
        geom = place["geometry"]
        position = Point(place['geometry']['coordinates'])
        name = place["properties"]["NAME"]
        pop = int(place["properties"]["POP_MAX"])
        cnt+=1
        if pop>25000 and position.within(ch):
            print(name, pop, position)
print(cnt)

In [ ]:
proj = ccrs.Mercator()

plt.figure(figsize=(15, 9))
ax = plt.axes(projection=proj)
ax.set_extent((ch.bounds[0], ch.bounds[2], ch.bounds[1], ch.bounds[3]), crs=ccrs.PlateCarree())
shape_feature = ShapelyFeature([ch], ccrs.PlateCarree(), facecolor='#EEFFFF', edgecolor='k')
ax.add_feature(shape_feature);

with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_populated_places') as c:
    for place in c:
        geom = place["geometry"]
        position = Point(place['geometry']['coordinates'])
        name = place["properties"]["NAME"]
        pop = int(place["properties"]["POP_MAX"])
        if pop>25000 and position.within(ch):
            ax.plot(position.x, position.y, color='blue', linewidth=2, marker='o', transform=ccrs.Geodetic())
            plt.text(position.x, position.y*1.0005, 
                 name, 
                 horizontalalignment='center', 
                 weight="regular", 
                 color="black", 
                 fontsize=10, 
                 transform=ccrs.Geodetic());